In [12]:
import importlib
import types

import yaml
import tensorflow as tf

# # %%
# def import_object_from_string(import_string):
#     import_string = import_string.strip()
#     module_name, object_name = '.'.join(import_string.split('.')[:-1]), import_string.split('.')[-1]
#     module = importlib.import_module(module_name, object_name)
#     return module.__getattribute__(object_name)

# # %%
# # class DatasetWrapper:
# #     def __init__(self):
# #         pass

    

# # %%
# class BiospecLoader(yaml.SafeLoader):
#     def __init__(self, *args, **kwargs):
#         super(BiospecLoader, self).__init__(*args, **kwargs)
        
#         # add constructors to Loader and associate with tag
#         self.add_constructor('!Source', self.constructor_source)
#         self.add_constructor('!Transform', self.constructor_transform)
    
#     def constructor_source(self, loader, node):
#         fields = fields = loader.construct_mapping(node, deep=True)
        
#         if 'args' not in fields:
#             fields['args'] = {}

#         source = import_object_from_string(fields['source'])(**fields['args'])

#         dataset = source()
#         return dataset

#     def constructor_transform(self, loader, node):
#         fields = fields = loader.construct_mapping(node, deep=True)

#         if 'args' not in fields:
#             fields['args'] = {}

#         transform = import_object_from_string(fields['transform'])
#         if isinstance(transform, types.FunctionType):
            

#         if not isinstance(fields['input'], tf.data.Dataset):
#             fields['input'] = tf.data.Dataset.zip(fields['input'], name='zip')
        
#         return fields['input'].map(transform(**fields['args']), name=transform.__name__)


# # %%
# def load_biospec(biospec_yaml, to_dataset=True):
#     with open(biospec_yaml, 'r') as f:
#         data = yaml.load(f, BiospecLoader)['data']
    
#     if to_dataset:
#         data = tf.data.Dataset.zip(data)

#     return data

# # %%
# data = load_biospec('biospec.yml', to_dataset=False)
# data

In [13]:
import pysam

# %%
base2int = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

# %%
baseComplement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}

# %%
def reverse_complement(dna_string):
    """Returns the reverse-complement for a DNA string."""

    complement = [baseComplement.get(base, 'N') for base in dna_string]
    reversed_complement = reversed(complement)
    return ''.join(list(reversed_complement))

# %%
def sequence2int(sequence, mapping=base2int):
    return [mapping.get(base, 999) for base in sequence]

# %%
def sequence2onehot(sequence, mapping=base2int):
    return tf.one_hot(sequence2int(sequence, mapping), depth=4)

In [ ]:
# from bioflow.io import pyfunc

In [14]:
import pandas as pd

class Bed():
    def __init__(self, filepath) -> None:
        
        self.bed_df = pd.read_csv(filepath, sep='\t', header=None)
        self.bed_df.columns = ['chrom', 'start', 'end', 'name', 'score', 'strand'] + [str(i) for i in range(6, len(self.bed_df.columns))]
        
    def __len__(self):
        return len(self.bed_df)
    
    # def __getitem__(self, key_idx):
    #     return self.fetch_one(**self.bed_df.loc[key_idx].to_dict())

    def __iter__(self):
        for i in range(0, len(self.bed_df)):
            yield self.bed_df.loc[i].to_dict()

bed = Bed('peaks/peaks.crosslink.slop-200b.bed')

In [ ]:
for row in bed:
    print(row)
    break

In [15]:
def make_signature(structure):
    structure_tensor = tf.nest.map_structure(tf.constant, structure)
    return tf.nest.map_structure(lambda x: x.dtype, structure_tensor)

def dataset_from_iterable(py_iterable):
    output_types = make_signature(next(iter(py_iterable)))
    return tf.data.Dataset.from_generator(lambda: iter(py_iterable), output_types=output_types)

bed_dataset = dataset_from_iterable(bed)
bed_dataset

2023-02-01 13:47:35.057144: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 13:47:35.057417: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-01 13:47:35.057462: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-02-01 13:47:35.057502: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-02-01 13:47:35.057540: W tensorflow/c

<FlatMapDataset element_spec={'chrom': TensorSpec(shape=<unknown>, dtype=tf.string, name=None), 'start': TensorSpec(shape=<unknown>, dtype=tf.int32, name=None), 'end': TensorSpec(shape=<unknown>, dtype=tf.int32, name=None), 'name': TensorSpec(shape=<unknown>, dtype=tf.string, name=None), 'score': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), 'strand': TensorSpec(shape=<unknown>, dtype=tf.string, name=None)}>

In [16]:
import tensorflow as tf

def tensor_to_numpy(tensor):
    return tensor.numpy()

def decode_if_bytes(x):
    if isinstance(x, bytes):
        x = x.decode('UTF-8')
    return x

def tensor_to_numpy_and_decode_if_bytes(x):
    x = tf.nest.map_structure(tensor_to_numpy, x)
    x = tf.nest.map_structure(decode_if_bytes, x)
    return x

def _dtype_to_tensor_spec(v):
    return tf.TensorSpec(None, v) if isinstance(v, tf.dtypes.DType) else v

def _tensor_spec_to_dtype(v):
  return v.dtype if isinstance(v, tf.TensorSpec) else v

def py_function_nest(func, inp, Tout, name=None):
    def wrapped_func(*flat_inp):
        reconstructed_inp = tf.nest.pack_sequence_as(inp, flat_inp, expand_composites=True)
        out = func(*reconstructed_inp)
        return tf.nest.flatten(out, expand_composites=True)

    flat_Tout = tf.nest.flatten(Tout, expand_composites=True)
    flat_out = tf.py_function(
        func=wrapped_func, 
        inp=tf.nest.flatten(inp, expand_composites=True),
        Tout=[_tensor_spec_to_dtype(v) for v in flat_Tout],
        name=name)
    spec_out = tf.nest.map_structure(_dtype_to_tensor_spec, Tout, expand_composites=True)
    out = tf.nest.pack_sequence_as(spec_out, flat_out, expand_composites=True)
    return out

def pyfunc(output_types, numpy=True, decode_bytes=True, expand_kwargs=True):
    def decorator(func):
        def func_wrapper(inp):
            if numpy:
                inp = tf.nest.map_structure(tensor_to_numpy, inp)
            if decode_bytes:
                inp = tf.nest.map_structure(decode_if_bytes, inp)
            
            if expand_kwargs:
                return func(**inp)
            else:
                return func(inp)
        return lambda x: py_function_nest(func_wrapper, inp=[x], Tout=output_types)
    return decorator

In [ ]:
class Fasta():
    def __init__(self, filepath) -> None:
        self._fasta = pysam.FastaFile(filepath)
    
    def fetch(self, chrom, start, end, strand='+', **kwargs):
        sequence = self._fasta.fetch(chrom, start, end)

        if strand == '+':
            pass
        elif strand == '-':
            sequence = ''.join(reverse_complement(sequence))
        else:
            raise ValueError(f'Unknown strand: {strand}')
        
        return sequence2onehot(sequence)

    def __call__(self, **kwargs):
        return self.fetch(**kwargs)

fasta = Fasta('/home/marc/data/GRCh38/grch38.fasta')


# @pyfunc(output_types = tf.float32)
# def fetch_fasta(chrom, start, end, **kwargs):
#     return fasta(chrom=chrom, start=start, end=end)

@pyfunc(output_types = tf.float32)
def fetch_fasta(**kwargs):
    return fasta(**kwargs)

fasta_dataset = bed_dataset.map(fetch_fasta)
print(next(iter(fasta_dataset)))

# fasta_dataset = bed_dataset.map(pyfunc(output_types=tf.float32)(lambda **kwargs: fasta(**kwargs)))
# print(next(iter(fasta_dataset)))

In [ ]:
class Fasta():
    def __init__(self, filepath) -> None:
        self._fasta = pysam.FastaFile(filepath)
    
    def fetch(self, chrom, start, end, strand='+', **kwargs):
        sequence = self._fasta.fetch(chrom, start, end)

        if strand == '+':
            pass
        elif strand == '-':
            sequence = ''.join(reverse_complement(sequence))
        else:
            raise ValueError(f'Unknown strand: {strand}')
        
        return sequence2onehot(sequence)

    def __call__(self, **kwargs):
        return self.fetch(**kwargs)

fasta = Fasta('/home/marc/data/GRCh38/grch38.fasta')


# @pyfunc(output_types = tf.float32)
# def fetch_fasta(chrom, start, end, **kwargs):
#     return fasta(chrom=chrom, start=start, end=end)

@pyfunc(output_types = tf.float32)
def fetch_fasta(**kwargs):
    return fasta(**kwargs)

# fasta_dataset = bed_dataset.map(fetch_fasta)
# print(next(iter(fasta_dataset)))

fasta_dataset = bed_dataset.map(pyfunc(output_types=tf.float32)(lambda **kwargs: fasta(**kwargs)))
print(next(iter(fasta_dataset)))

In [40]:
# def new_pyfunc(Tout, numpy=True, decode_bytes=True, expand_kwargs=True):
#     def decorator(func):
#         # func = lambda **kwargs: func(**kwargs)
#         def func_wrapper(inp):
#             print(inp)
#             if numpy:
#                 inp = tf.nest.map_structure(tensor_to_numpy, inp)
#             if decode_bytes:
#                 inp = tf.nest.map_structure(decode_if_bytes, inp)
            
#             if expand_kwargs:
#                 return func(**inp)
#             else:
#                 return func(inp)
#         return lambda x: py_function_nest(func_wrapper, inp=[x], Tout=Tout)
#     return decorator

def better_py_function_kwargs(Tout, numpy=True, decode_bytes=True):
    def decorator(func):
        def func_wrapper(kwargs):
            if numpy:
                kwargs = tf.nest.map_structure(tensor_to_numpy, kwargs)
            if decode_bytes:
                kwargs = tf.nest.map_structure(decode_if_bytes, kwargs)
            return func(**kwargs)
        return lambda kwargs: py_function_nest(func_wrapper, inp=[kwargs], Tout=Tout)
    return decorator

class Fasta():
    def __init__(self, filepath) -> None:
        self._fasta = pysam.FastaFile(filepath)
    
    def fetch(self, chrom, start, end, strand='+', **kwargs):
        sequence = self._fasta.fetch(chrom, start, end)

        if strand == '+':
            pass
        elif strand == '-':
            sequence = ''.join(reverse_complement(sequence))
        else:
            raise ValueError(f'Unknown strand: {strand}')
        
        return sequence2onehot(sequence)

    def __call__(self, kwargs):
        return better_py_function_kwargs(Tout=tf.float32)(self.fetch)(kwargs)

fasta = Fasta('/home/marc/data/GRCh38/grch38.fasta')


fasta_dataset = bed_dataset.map(fasta)
print(next(iter(fasta_dataset)))

tf.Tensor(
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]

In [ ]:
f = fasta.__call__
f

In [ ]:
f()

In [ ]:
# @pyfunc(output_types = tf.float32)
# def fetch_fasta(chrom, start, end, **kwargs):
#     return fasta(chrom=chrom, start=start, end=end)

# @pyfunc(output_types = tf.float32)
# def fetch_fasta(**kwargs):
#     return fasta(**kwargs)

# fasta_dataset = bed_dataset.map(fetch_fasta)
# print(next(iter(fasta_dataset)))

fasta_dataset = bed_dataset.map(fasta)
print(next(iter(fasta_dataset)))

In [ ]:
# from bioflow.io.transforms.fasta import Fasta

# fasta = Fasta('/home/marc/data/GRCh38/grch38.fasta')

# @pyfunc(output_types=tf.float16, expand_kwargs=True)
# def func_wrapper(chrom, start, end):
#     # return tf.constant(2, dtype=tf.float16)
#     return fasta.fetch(chrom=chrom, start=start, end=end)

@pyfunc(output_types = tf.float32)
def fetch_fasta(chrom, start, end, **kwargs):
    return fasta(chrom=chrom, start=start, end=end)

fasta_dataset = dataset.map(fetch_fasta)
next(iter(fasta_dataset))

In [ ]:
row_py = tf.nest.map_structure(lambda x: x.numpy(), next(iter(dataset)))
row_py

In [ ]:
row_py['chrom'].decode('UTF-8')

In [ ]:
def decode_if_string(x):
    if isinstance(x, bytes):
        x = x.decode('UTF-8')
    return x

In [ ]:
def fetch_fasta(x):
    x = tf.numpy_function(lambda x: x['start'], inp=[x], Tout=tf.int32)
    print(x)
    return x

In [ ]:
# new_dataset = dataset.map(fetch_fasta)
# next(iter(new_dataset))

In [ ]:
tf.constant(2).dtype

In [ ]:
bed = Bed('peaks/peaks.crosslink.slop-200b.bed')

output_types = make_signature(next(iter(bed)))

dataset = tf.data.Dataset.from_generator(lambda: iter(bed), output_types=output_types)

In [ ]:
# def _dtype_to_tensor_spec(v):
#     return tf.TensorSpec(None, v) if isinstance(v, tf.dtypes.DType) else v

# def _tensor_spec_to_dtype(v):
#   return v.dtype if isinstance(v, tf.TensorSpec) else v

# def new_py_function(func, inp, Tout, name=None):
#     def wrapped_func(*flat_inp):
#         reconstructed_inp = tf.nest.pack_sequence_as(inp, flat_inp, expand_composites=True)
#         out = func(*reconstructed_inp)
#         return tf.nest.flatten(out, expand_composites=True)

#     flat_Tout = tf.nest.flatten(Tout, expand_composites=True)
#     flat_out = tf.py_function(
#         func=wrapped_func, 
#         inp=tf.nest.flatten(inp, expand_composites=True),
#         Tout=[_tensor_spec_to_dtype(v) for v in flat_Tout],
#         name=name)
#     spec_out = tf.nest.map_structure(_dtype_to_tensor_spec, Tout, expand_composites=True)
#     out = tf.nest.pack_sequence_as(spec_out, flat_out, expand_composites=True)
#     return out

# dataset = tf.data.Dataset.from_tensor_slices([0, 1, 2])
# dataset = dataset.map(lambda x: {'a': x, 'b': x})
# print(dataset, '\n')

# dataset = dataset.map(lambda x: new_py_function(identity, inp=[x], Tout={'a': tf.int32}))
# print(dataset, '\n')

# print(next(iter(dataset)))

In [ ]:
import tensorflow as tf

def tensor_to_numpy(tensor):
    return tensor.numpy()

def decode_if_bytes(x):
    if isinstance(x, bytes):
        x = x.decode('UTF-8')
    return x

def tensor_to_numpy_and_decode_if_bytes(x):
    x = tf.nest.map_structure(tensor_to_numpy, x)
    x = tf.nest.map_structure(decode_if_bytes, x)
    return x

def _dtype_to_tensor_spec(v):
    return tf.TensorSpec(None, v) if isinstance(v, tf.dtypes.DType) else v

def _tensor_spec_to_dtype(v):
  return v.dtype if isinstance(v, tf.TensorSpec) else v

def py_function_nest(func, inp, Tout, name=None):
    def wrapped_func(*flat_inp):
        reconstructed_inp = tf.nest.pack_sequence_as(inp, flat_inp, expand_composites=True)
        out = func(*reconstructed_inp)
        return tf.nest.flatten(out, expand_composites=True)

    flat_Tout = tf.nest.flatten(Tout, expand_composites=True)
    flat_out = tf.py_function(
        func=wrapped_func, 
        inp=tf.nest.flatten(inp, expand_composites=True),
        Tout=[_tensor_spec_to_dtype(v) for v in flat_Tout],
        name=name)
    spec_out = tf.nest.map_structure(_dtype_to_tensor_spec, Tout, expand_composites=True)
    out = tf.nest.pack_sequence_as(spec_out, flat_out, expand_composites=True)
    return out


def pyfunc(output_types, numpy=True, decode_bytes=True, expand_kwargs=True):
    def decorator(func):
        def func_wrapper(inp):
            if numpy:
                inp = tf.nest.map_structure(tensor_to_numpy, inp)
            if decode_bytes:
                inp = tf.nest.map_structure(decode_if_bytes, inp)
            
            if expand_kwargs:
                return func(**inp)
            else:
                return func(inp)
        return lambda x: py_function_nest(func_wrapper, inp=[x], Tout=output_types)
    return decorator


dataset = tf.data.Dataset.from_tensor_slices([0, 1, 2])
dataset = dataset.map(lambda x: {'a': x, 'b': x})
print(dataset, '\n')

# def kwargs_identity(a, **kwargs):
#   print(a)
#   return {'a': a+7}

# dataset = dataset.map(pyfunc(kwargs_identity, output_types={'a': tf.int32}))
# print(dataset, '\n')

@pyfunc(output_types={'z': tf.int32})
def kwargs_identity_to_dict(**kwargs):
    return {'z': kwargs['a']+14}

@pyfunc(output_types=tf.int32)
def kwargs_identity_to_tensor(a, **kwargs):
    return a+14


dataset = dataset.map(kwargs_identity_to_dict)
print(dataset, '\n')

print(next(iter(dataset)))

In [ ]:
kwargs_identity_to_tensor.__

In [ ]:
bed_dataset = dataset_from_iterable(bed)
# print(next(iter(bed_dataset)))

@pyfunc(output_types = tf.float32)
def fetch_fasta(chrom, start, end, **kwargs):
    return fasta(chrom=chrom, start=start, end=end)

@pyfunc(output_types = tf.float32, expand_kwargs=False)
def fetch_fasta_prefilled(x):
    return fasta(chrom='chr1', start=10, end=20)

fasta_dataset = bed_dataset.map(fetch_fasta_prefilled)
print(next(iter(fasta_dataset)))

In [ ]:
@pyfunc(output_types = tf.float32)
def fetch_fasta(chrom, start, end, **kwargs):
    return fasta(chrom=chrom, start=start, end=end)

# fetch_fasta('chr1', 10, 20)

In [ ]:
fasta_dataset = bed_dataset.map(fetch_fasta)
print(fasta_dataset)

In [ ]:

def decorator_factory(number):
    def decorator(fun):
        def wrapper(*args, **kwargs):
            x = fun(*args, **kwargs)
            return x*number
        return wrapper
    return decorator

@decorator_factory(number=8)
def seven():
    return 7

def six():
    return 6

print(seven())
print(decorator_factory(number=2)(six)())

In [ ]:
f = pyfunc_wrapper(lambda a, **kwargs: {'a': a['a']}, output_types={'a': tf.int32})

In [ ]:
input